
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

Cloning into 'tf-levenberg-marquardt'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 53 (delta 31), reused 29 (delta 14), pack-reused 0
Receiving objects: 100% (53/53), 32.99 KiB | 16.50 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y3", label="oxigenio")

/tmp/ipykernel_33913/160627683.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-20 18:54:31.220678: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 18:54:31.265790: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 18:54:31.265840: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 18:54:31.267124: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-20 18:54:31.274566: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 18:54:31.275519: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[25, 10], [30, 15], [45, 20]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02, 0.1])

Testando combinacao1: Hidden Size=[25, 10], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  6.011006775000169
Stopped at epoch:  161
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.20650340699967273
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.28092265999976007
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.24745896600006745
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.21858332000010705
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.18666786100038735
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 7 +++

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_0_0,0.846770,-0.179377,0.689329,0.470540,0.169580,0.588902,0.450963,0.523989
model_1_0_1,0.847105,-0.180954,0.689718,0.470388,0.169210,0.589690,0.450397,0.524140
model_1_0_2,0.847422,-0.182518,0.690086,0.470224,0.168859,0.590471,0.449863,0.524303
model_1_0_3,0.847720,-0.184100,0.690441,0.470046,0.168529,0.591260,0.449348,0.524478
model_1_0_4,0.848002,-0.185686,0.690775,0.469853,0.168217,0.592053,0.448863,0.524669
model_1_0_5,0.848268,-0.187253,0.691093,0.469654,0.167923,0.592835,0.448402,0.524866
model_1_0_6,0.848519,-0.188784,0.691383,0.469445,0.167645,0.593599,0.447981,0.525073
model_1_0_7,0.848754,-0.190345,0.691667,0.469224,0.167385,0.594379,0.447569,0.525292
model_1_0_8,0.848975,-0.191890,0.691930,0.468993,0.167140,0.595150,0.447186,0.525520
model_1_0_9,0.849185,-0.193419,0.692188,0.468763,0.166908,0.595914,0.446813,0.525748


DataFrame salvo em ../results/metrics_1_0
+++++++++++ [1_1] | 1 ++++++++++++++++++
Elapsed time:  0.38268005000009
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_1] | 2 ++++++++++++++++++
Elapsed time:  0.21200069799988341
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_1] | 3 ++++++++++++++++++
Elapsed time:  0.26542828199990254
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_1] | 4 ++++++++++++++++++
Elapsed time:  0.24243383100019855
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [1_1] | 5 ++++++++++++++++++
Elapsed time:  0.30121278999968126
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_1] | 6 ++++++++++++++++++
Elapsed time:  0.9423610599997119
Stopped at epoch:  28
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_1] | 7 ++++++++++++++++++
Elapsed time:  0.211242123

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_1_1,0.800313,0.520942,-0.090768,0.290867,0.220995,0.454102,0.745131,0.591056
model_1_1_2,0.802122,0.522981,-0.090433,0.292223,0.218993,0.452169,0.744902,0.589926
model_1_1_3,0.803707,0.524453,-0.091065,0.292865,0.217239,0.450774,0.745334,0.589391
model_1_1_4,0.805210,0.525845,-0.091819,0.293413,0.215576,0.449454,0.745849,0.588934
model_1_1_5,0.806624,0.527714,-0.094032,0.293686,0.214010,0.447683,0.747360,0.588707
model_1_1_6,0.807796,0.528423,-0.096575,0.293131,0.212714,0.447011,0.749098,0.589170
model_1_1_7,0.808822,0.528780,-0.099727,0.292131,0.211577,0.446672,0.751251,0.590003
model_1_1_8,0.809775,0.529052,-0.102812,0.291103,0.210523,0.446415,0.753358,0.590859
model_1_1_9,0.810465,0.528355,-0.106594,0.289226,0.209759,0.447075,0.755942,0.592424
model_1_1_10,0.811175,0.527841,-0.109873,0.287652,0.208974,0.447562,0.758182,0.593736


DataFrame salvo em ../results/metrics_1_1
+++++++++++ [1_2] | 1 ++++++++++++++++++
Elapsed time:  0.27098568600013095
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_2] | 2 ++++++++++++++++++
Elapsed time:  0.23436063099961757
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_2] | 3 ++++++++++++++++++
Elapsed time:  0.2657663329996467
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_2] | 4 ++++++++++++++++++
Elapsed time:  0.2278496140002062
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_2] | 5 ++++++++++++++++++
Elapsed time:  0.21496921200014185
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_2] | 6 ++++++++++++++++++
Elapsed time:  0.1964490969999133
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_2] | 7 ++++++++++++++++++
Elapsed time:  0.297481305

...

DataFrame salvo em ../results/metrics_12_2
+++++++++++ [12_3] | 1 ++++++++++++++++++
Elapsed time:  1.1614665639999657
Stopped at epoch:  11
1/1 [==============================] - 0s 30ms/step
+++++++++++ [12_3] | 2 ++++++++++++++++++
Elapsed time:  0.7689458049999303
Stopped at epoch:  10
1/1 [==============================] - 0s 18ms/step
+++++++++++ [12_3] | 3 ++++++++++++++++++
Elapsed time:  0.8346886340004858
Stopped at epoch:  13
1/1 [==============================] - 0s 23ms/step
+++++++++++ [12_3] | 4 ++++++++++++++++++
Elapsed time:  0.4890015480004877
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [12_3] | 5 ++++++++++++++++++
Elapsed time:  0.6047767950003617
Stopped at epoch:  6
1/1 [==============================] - 0s 25ms/step
+++++++++++ [12_3] | 6 ++++++++++++++++++
Elapsed time:  0.5196410919998016
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [12_3] | 7 ++++++++++++++++++
Elapsed time:  0.5

2024-03-20 19:23:43.705501: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  0.4876680489996943
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [12_3] | 11 ++++++++++++++++++
Elapsed time:  0.4112404659999811
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [12_3] | 12 ++++++++++++++++++
Elapsed time:  0.5656985330006137
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [12_3] | 13 ++++++++++++++++++
Elapsed time:  0.5074450259999139
Stopped at epoch:  6
1/1 [==============================] - 0s 28ms/step
+++++++++++ [12_3] | 14 ++++++++++++++++++
Elapsed time:  0.5172337819994937
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [12_3] | 15 ++++++++++++++++++
Elapsed time:  0.48856649299978017
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [12_3] | 16 ++++++++++++++++++
Elapsed time:  0.4460657049994552
Stopped at epoch:  6
1/1 [==============================] - 0s 29

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_12_3_0,0.80933,0.353841,0.913794,0.585582,0.211016,1.103999,0.104977,0.633871


DataFrame salvo em ../results/metrics_12_3
+++++++++++ [12_4] | 1 ++++++++++++++++++
Elapsed time:  0.059686274000341655
Stopped at epoch:  0
1/1 [==============================] - 0s 19ms/step
+++++++++++ [12_4] | 2 ++++++++++++++++++
Elapsed time:  0.04960984199988161
Stopped at epoch:  0
1/1 [==============================] - 0s 22ms/step
+++++++++++ [12_4] | 3 ++++++++++++++++++
Elapsed time:  0.04494209399945248
Stopped at epoch:  0
1/1 [==============================] - 0s 22ms/step
+++++++++++ [12_4] | 4 ++++++++++++++++++
Elapsed time:  0.055187893999573134
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [12_4] | 5 ++++++++++++++++++
Elapsed time:  0.03485127099975216
Stopped at epoch:  0
1/1 [==============================] - 0s 23ms/step
+++++++++++ [12_4] | 6 ++++++++++++++++++
Elapsed time:  0.05807616899983259
Stopped at epoch:  0
1/1 [==============================] - 0s 17ms/step
+++++++++++ [12_4] | 7 ++++++++++++++++++
Elapsed time: